In [ ]:
# Cell 1: Bulk Download All Monthly Climate Data with Verification (1981-2016)
import cdsapi
import os
import time
import xarray as xr

# --- Configuration ---
# A dedicated folder for the full monthly dataset
output_dir = '../data/climate_monthly_full/'
os.makedirs(output_dir, exist_ok=True)

# The full range of years for our analysis
years_to_download = range(1981, 2017)

# The variables we want to download
variables_to_download = [
    '2m_temperature', 'total_precipitation', 'volumetric_soil_water_layer_1',
    'surface_net_solar_radiation', 'potential_evaporation',
]

# --- API Request Loop ---
c = cdsapi.Client()

print(f"Starting robust bulk download for {len(years_to_download)} years.")
print(f"Files will be saved in: {output_dir}")

for year in years_to_download:
    output_file = os.path.join(output_dir, f'era5_land_monthly_{year}.grib')
    
    # This check makes the script resumable if it gets interrupted
    if os.path.exists(output_file):
        print(f"File for year {year} already exists. Skipping.")
        continue

    print(f"\n--------------------------------------------------")
    print(f"Submitting request for year: {year}")
    
    try:
        request_dictionary = {
            'product_type': 'monthly_averaged_reanalysis',
            'variable': variables_to_download,
            'year': str(year),
            'month': [f'{m:02d}' for m in range(1, 13)],
            'time': '00:00',
            'format': 'grib',
            'download_format': 'unarchived',
        }

        # --- DOWNLOAD STEP ---
        c.retrieve(
            'reanalysis-era5-land-monthly-means',
            request_dictionary,
            output_file
        )
        print(f"Download for year {year} complete! File size: {os.path.getsize(output_file) / 1e6:.2f} MB")
        
        # --- VERIFICATION STEP ---
        print(f"  -> Verifying file integrity for {year}...")
        try:
            # Attempt to open the file we just downloaded, checking for one variable
            ds_check = xr.open_dataset(
                output_file,
                engine="cfgrib",
                backend_kwargs={"indexpath": "", "filter_by_keys": {"shortName": "2t"}}
            )
            print(f"  -> SUCCESS: File for {year} is valid.")
        except Exception as e:
            print(f"  -> !!! FAILED VERIFICATION: File for {year} is corrupted or unreadable. Deleting it.")
            print(f"     Error was: {e}")
            os.remove(output_file) # Delete the bad file so we can try again later

        # A small polite pause between requests
        time.sleep(2)

    except Exception as e:
        print(f"\nAn error occurred during download for year {year}: {e}")
        print("The script will continue with the next year.")
        continue

print("\n--- All downloads finished! ---")